In [16]:
import matplotlib.pyplot as plt
import numpy as np
import random
import time
import IPython

In [17]:
import seaborn as sns
import pylab as pl

In [34]:
class VacuumCleaner():
    def __init__(self, size=5, sym_empty='□ ', sym_trash='* ',
                 sym_vacuum='▩ '):
        self.size = size
        self.sym_empty = sym_empty
        self.sym_trash = sym_trash
        self.sym_vacuum = sym_vacuum
        self.string_mapping = np.vectorize(lambda s: self.sym_empty if s == 0
                                    else self.sym_trash if s == 1
                                    else self.sym_vacuum)
        
        self.field = np.zeros((self.size, self.size))
        
        self.v_path = np.zeros((self.size, self.size))
        self.v_path[1:(self.size - 1), 1] = np.ones((self.size - 2))
        self.v_path[1:(self.size - 1), (self.size - 2)] = np.ones((self.size \
                                                                   - 2))
        self.v_path[1, 1:(self.size - 1)] = np.ones((self.size - 2))
        self.v_path[(self.size - 2), 1:(self.size - 1)] = np.ones((self.size \
                                                                   - 2))
        
        self.v_path_indexes = np.array(np.where(self.v_path == 1)).transpose()

        self.directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]
        
        self.speed = 0.1
        self.style = 'print'
        
    def output_iteration(self, i): # 'print' or 'plot'
        IPython.display.clear_output(wait=True)
        print('Iteration {:>4d} '.format(i + 1))
        self.calculate_trash()
        print('Trash on the field in percent: {:.1f}%'.format(self.trash * 100))
        if self.style == 'print':
            self.print_field()
        elif self.style == 'plot':
            #if i % (self.iterations / 10) == 0:
            self.draw_field()
        time.sleep(self.speed)
    
    def print_field(self):
        print('\n'.join(''.join(str(cell) for cell in row)
                        for row in self.string_mapping(self.field)))
    
    def draw_field(self):
        pl.clf()
        sns.heatmap(self.field, cbar=False)
        IPython.display.display(pl.gcf())
        return
    
    def calculate_trash(self):
        self.trash = (self.field == 1).sum() / self.field.size
    
    def check_near_trash(self):
        np.random.shuffle(self.directions)
        
        self.stepped = False
        min_distance = self.size * self.size
        #print('current:', self.v_pos)
        
        for direction in self.directions:
            possible_v_pos = tuple(np.array(self.v_pos) + np.array(direction))
            if possible_v_pos[0] in range(self.size) \
            and possible_v_pos[1] in range(self.size) \
            and self.field[possible_v_pos] == 1:
                self.field[self.v_pos] = 0
                self.field[possible_v_pos] = 2
                self.v_pos = possible_v_pos
                self.stepped =  True
                break
                
            if np.sort(np.linalg.norm(abs(self.v_path_indexes - \
                                      possible_v_pos), axis = 1))[0] < min_distance:
                min_distance = np.sort(np.linalg.norm(\
                    abs(self.v_path_indexes - possible_v_pos), axis = 1))[0]
                self.best_v_pos = possible_v_pos
                
    def is_vacuum_on_his_path(self):
        return((self.v_path * self.field == 2).any())
    
    def run_process(self, iterations=100, speed=0.1,
                    path_type='right', style='print'):
        self.iterations = iterations
        self.speed = speed
        self.style = style
        if self.style == 'plot':
            pl.rcParams['figure.figsize'] = self.size / 2, self.size / 2
            
        self.v_pos = (1, 1)
        self.field[self.v_pos] = 2
            
        for i in range(self.iterations):
            # throw trash randomly 
            if np.random.rand() > 0.5:
                index = np.random.randint(self.size, size=2)
                if self.field[tuple(index)] != 2:
                    self.field[tuple(index)] = 1
                
            # output
            self.output_iteration(i)
            
            # check trash
            # move and clear if trash is near
            self.check_near_trash()
            
            # move to/along vacuum path if there isn't any trash
            if not self.stepped:
                if path_type == 'random':
                    np.random.shuffle(self.directions)
                    self.best_v_pos = tuple(np.abs(np.array(self.v_pos) + \
                                                   np.array(directions[0])))
                self.field[self.v_pos] = 0
                self.field[self.best_v_pos] = 2
                self.v_pos = self.best_v_pos
            
            # output
            self.output_iteration(i)

In [35]:
v = VacuumCleaner(5)
v.run_process(iterations=1000, speed=0, path_type='right', style='print')
#pl.clf()

Iteration 1000 
Trash on the field in percent: 24.0%
* □ □ □ □ 
□ □ □ □ * 
▩ □ □ □ □ 
* □ □ * □ 
□ * * □ □ 
